# Sets

[Click here to run this chapter on Colab](https://colab.research.google.com/github/AllenDowney/DSIRP/blob/main/notebooks/set.ipynb)

## Set operators and methods

The following example is based on Luciano Ramalho's talk, [Set Practice: Learning from Python's set type](https://www.youtube.com/watch?v=tGAngdU_8D8).

In [1]:
def fibonacci(stop):
    a, b = 0, 1
    while a < stop:
        yield a
        a, b = b, a + b

In [2]:
f = {n for n in fibonacci(10)}
f

{0, 1, 2, 3, 5, 8}

In [3]:
def primes(stop):
    m = {}
    q = 2
    while q < stop:
        if q not in m:
            yield q 
            m[q*q] = [q]
        else:
            for p in m[q]:
                m.setdefault(p+q, []).append(p)
            del m[q]
        q += 1

In [4]:
p = {n for n in primes(10)}
p

{2, 3, 5, 7}

Checking membership is constant time.

In [5]:
8 in f

True

In [6]:
8 in p

False

Intersection is like AND: it returns elements in f AND in p.

In [7]:
f & p

{2, 3, 5}

Union is like OR: it returns elements in f OR in p.

In [8]:
f | p

{0, 1, 2, 3, 5, 7, 8}

Symmetric difference is like XOR: elements from `f` OR `p` but not both.

In [9]:
f ^ p

{0, 1, 7, 8}

Here are the Fibonacci numbers that are not prime.

In [10]:
f - p

{0, 1, 8}

And the primes that are not Fibonacci numbers.

In [11]:
p - f

{7}

The comparison operators check for subset and superset relationships.

The Fibonacci numbers are not a superset of the primes.

In [12]:
f >= p

False

And the primes are not a superset of the Fibonacci numbers.

In [13]:
p >= f

False

In that sense, sets are not like numbers: they are only [partially ordered](https://en.wikipedia.org/wiki/Partially_ordered_set).

`f` is a superset of `{1, 2, 3}`

In [14]:
f >= {1, 2, 3}

True

In [15]:
p >= {1, 2, 3}

False

Sets provide methods as well as operators. Why?

For one thing, the argument you pass to a method can be any iterable, not just a set.

In [16]:
try:
    f >= [1, 2, 3]
except TypeError as e:
    print(e)

'>=' not supported between instances of 'set' and 'list'


In [17]:
f.issuperset([1,2,3])

True

Methods also accept more than one argument:

In [18]:
f.union([1,2,3], (3,4,5), {5,6,7}, {7:'a', 8:'b'})

{0, 1, 2, 3, 4, 5, 6, 7, 8}

If you don't have a set to start with, you can use an empty set.

In [19]:
set().union([1,2,3], (3,4,5), {5,6,7}, {7:'a', 8:'b'})

{1, 2, 3, 4, 5, 6, 7, 8}

One small syntax nuisance: `{1, 2, 3}` is a set, but `{}` is an empty dictionary.

## Spelling Bee

[The New York Times Spelling Bee](https://www.nytimes.com/puzzles/spelling-bee) is a daily puzzle where the goal is to spell as many words as possible using only the given set of seven letters. 
For example, in a recent Spelling Bee, the available letters were `dehiklo`,
so you could spell "like" and "hold".

You can use each of the letters more than once, so "hook" and "deed" would be allowed, too.

To make it a little more interesting, one of the letters is special and must be included in every word.
In this example, the special letter is `o`, so "hood" would be allowed, but not "like".

Each word you find scores points depending on it's length, which must be at least four letters.
A word that uses all of the letters is called a "pangram" and scores extra points.

We'll use this puzzle to explore the use of Python sets.

Suppose we're given a word and we would like to know whether it can be spelled using only a given set of letters.
The following function solves this problem using string operations.

In [20]:
def uses_only(word, letters):
    for letter in word:
        if letter not in letters:
            return False
    return True

If we find any letters in `word` that are not in the list of letters, we can return `False` immediately.
If we get through the word without finding any unavailable letters, we can return `True`.

Let's try it out with some examples. In a recent Spelling Bee, the available letters were `dehiklo`.
Let's see what we can spell with them.

In [21]:
letters = "dehiklo"
uses_only('lode', letters)

True

In [22]:
uses_only('implode', letters)

False

**Exercise:** It is possible to implement `uses_only` more concisely using set operations rather than list operations. [Read the documentation of the `set` class](https://docs.python.org/3/tutorial/datastructures.html#sets) and rewrite `uses_only` using sets.

In [31]:
def uses_only(word, letters):
    return set(word)&set(letters)==set(word)

uses_only('racecar','acre')

True

In [32]:
uses_only('lode', letters)

True

In [33]:
uses_only('implode', letters)

False

## Word list

The following function downloads a list of about 100,000 words in American English.

In [34]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/DSIRP/raw/main/american-english')

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)>

The file contains one word per line, so we can read the file and split it into a list of words like this:

In [37]:
word_list = open('../american-english').read().split()
len(word_list)

102401

**Exercise:** Write a loop that iterates through this word list and prints only words 

* With at least four letters,

* That can be spelled using only the letters `dehiklo`, and

* That include the letter `o`.

In [42]:
for word in word_list:
    if ('o' in word) and (len(word)>=4) and (uses_only(word,'dehiklo')):
        print(word)

diode
dodo
dole
doled
doll
dolled
doodle
doodled
hellhole
hello
hoed
hold
hole
holed
hood
hooded
hoodie
hoodoo
hoodooed
hook
hooked
idol
kiddo
kilo
kook
kookie
likelihood
lode
loll
lolled
look
looked
oiled
oldie
oleo


**Exercise:** Now let's check for pangrams. 
Write a function called `uses_all` that takes two strings and returns `True` if the first string uses all of the letters in the second string.
Think about how to express this computation using set operations.

Test your function with at least one case that returns `True` and one that returns `False`.

In [44]:
def uses_all(word, letters):
    return set(word)&set(letters)==set(letters)

In [45]:
uses_all('minecraft','race')

True

In [46]:
uses_all('chaos','hack')

False

**Exercise:** Modify the previous loop to search the word list for pangrams using `uses_only` and `uses_all`.

Or, as a bonus, write a function called `uses_all_and_only` that checks both conditions using a single `set` operation.

In [47]:
def uses_all_and_only(word, letters):
    return uses_all(word, letters) and uses_only(word, letters)

In [48]:
uses_all_and_only('undertale','deltarune')

True

In [49]:
uses_all_and_only('football','labfo')

False

## Leftovers

So far we've been writing Boolean functions that test specific conditions, but if they return `False`, they don't explain why.
As an alternative to `uses_only`, we could write a function called `bad_letters` that takes a word and a set of letters, and returns a new string that contains the letters in words that are not available.  Let's call it `bad_letters`. 

In [35]:
def bad_letters(word, letters):
    return set(word) - set(letters)

Now if we run this function with an illegal word, it will tell us which letters in the word are not available.

In [36]:
bad_letters('oilfield', letters)

**Exercise:** Write a function called `unused_letters` that takes a word and a set of letters and returns the subset of the letters that are not used in `word`.

**Exercise:** Write a function called `no_duplicates` that takes a string and returns `True` if each letter appears only once.

*Data Structures and Information Retrieval in Python*

Copyright 2021 Allen Downey

License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)